In [8]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
matplotlib.use('agg')
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing as pp
from keras.utils import np_utils
ss = pp.StandardScaler()
import h5py

X_train = np.load("20170827_mawilab_flow_000flags_nosvc_features.pkl")
Y_train = np.load("20170827_mawilab_flow_000flags_nosvc_labels.pkl")


# Test Set 2 : 20170827_mawilab_flow_002
X_test2 = np.load("20170827_mawilab_flow_002flags_nosvc_features.pkl")
Y_test2 = np.load("20170827_mawilab_flow_002flags_nosvc_labels.pkl")
X_train = ss.fit_transform(X_train)
X_test2 = ss.fit_transform(X_test2)



# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test2 = np.reshape(X_test2, (X_test2.shape[0],X_test2.shape[1],1))


In [9]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(15, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(2),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 15, 64)            256       
_________________________________________________________________
activation_7 (Activation)    (None, 15, 64)            0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 7, 64)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 448)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               57472     
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
__________

In [10]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn1d_1.fit(X_train, Y_train, batch_size=64, validation_data=(X_test2, Y_test2) ,epochs=10)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("20170827_mawilab_flow_000.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("20170827_mawilab_flow_000.h5")
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_Valaccuaracy00_02.png')

Train on 38361 samples, validate on 47265 samples
Epoch 1/10
38361/38361 [==============================] - 12s 303us/step - loss: 0.4810 - acc: 0.7045 - val_loss: 0.4874 - val_acc: 0.7155
Epoch 2/10
38361/38361 [==============================] - 9s 239us/step - loss: 0.4631 - acc: 0.7189 - val_loss: 0.4754 - val_acc: 0.7127
Epoch 3/10
38361/38361 [==============================] - 9s 231us/step - loss: 0.4593 - acc: 0.7196 - val_loss: 0.4742 - val_acc: 0.7172
Epoch 4/10
38361/38361 [==============================] - 9s 241us/step - loss: 0.4568 - acc: 0.7238 - val_loss: 0.4698 - val_acc: 0.7214
Epoch 5/10
38361/38361 [==============================] - 9s 233us/step - loss: 0.4547 - acc: 0.7265 - val_loss: 0.4629 - val_acc: 0.7208
Epoch 6/10
38361/38361 [==============================] - 9s 233us/step - loss: 0.4523 - acc: 0.7275 - val_loss: 0.4671 - val_acc: 0.7204
Epoch 7/10
38361/38361 [==============================] - 9s 233us/step - loss: 0.4505 - acc: 0.7293 - val_loss: 0.4696 -

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score
y_pred = cnn1d_1.predict(X_test2)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

y_pred = cnn1d_1.predict_classes(X_test2)
print(y_pred)

p=cnn1d_1.predict_proba(X_test2)

target_names = ['normal', 'anomaly']
print(classification_report(np.argmax(Y_test2, axis=1), y_pred, target_names=target_names, digits=4))
print(confusion_matrix(np.argmax(Y_test2, axis=1), y_pred))
Y_test2 = np.argmax(Y_test2, axis=1)

print(f1_score(Y_test2, y_pred, average='macro'))

print(f1_score(Y_test2, y_pred, average='micro'))

print(f1_score(Y_test2, y_pred, average='weighted'))

print(f1_score(Y_test2, y_pred, average=None))

[[0.49614394 0.50520635]
 [0.30483556 0.69491464]
 [0.00258263 0.99742836]
 ...
 [0.49614394 0.50520635]
 [0.14639966 0.8506357 ]
 [0.00258263 0.99742836]]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
             precision    recall  f1-score   support

     normal     0.6560    0.3711    0.4740     14552
    anomaly     0.7655    0.9134    0.8330     32713

avg / total     0.7318    0.7465    0.7225     47265

[[ 5400  9152]
 [ 2832 29881]]
0.6534915758526412
0.7464508621601608
0.7224525496131438
[0.47401685 0.8329663 ]
